In [7]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [8]:
import os
import re

import numpy as np
import pandas as pd
import pytest

from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model
from skforecast.deep_learning.utils import _create_and_compile_model_exog, _create_and_compile_model_no_exog

# Export torch keras backend
os.environ["KERAS_BACKEND"] = "torch"
import keras
from keras.optimizers import Adam
from keras.losses import MeanSquaredError

In [9]:
series = pd.DataFrame(
    {
        "1": pd.Series(np.arange(50)),
        "2": pd.Series(np.arange(50)),
        "3": pd.Series(np.arange(50)),
    }
)
model = create_and_compile_model(
            series=series, 
            levels=["1", "2"],    
            lags=3,           
            steps=4,              
            recurrent_layer="LSTM",
            recurrent_units=100,
            recurrent_layers_kwargs={"activation": "relu"},
            dense_units=[128, 64],
            dense_layers_kwargs={"activation": "relu"},
            output_dense_layer_kwargs={"activation": "linear"},
            compile_kwargs={"optimizer": Adam(learning_rate=0.01), "loss": MeanSquaredError()},
        )

keras version: 3.10.0
Using backend: tensorflow
tensorflow version: 2.19.0



In [14]:
forecaster = ForecasterRnn(model, levels=["1", "2"], lags=3)
forecaster.fit(series, store_in_sample_residuals=True)

# Call the predict method
int_preds = forecaster.predict_interval()

# Check the shape and values of the predictions
assert int_preds.shape == (4 * len(["1", "2"]), 4)
int_preds

/home/joaquin/miniconda3/envs/skforecast_17_py12/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 20 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.3296


,level,pred,lower_bound,upper_bound
50,1,11.903735,-14.386130,38.193600
50,2,20.846828,-12.778509,54.472165
51,1,26.605282,0.315417,52.895147
51,2,-3.041294,-36.666631,30.584043
52,1,18.407512,-7.882353,44.697378
52,2,13.654057,-19.971281,47.279394
53,1,28.990269,2.700404,55.280134
53,2,25.379845,-8.245492,59.005182


In [15]:

forecaster.predict_interval(steps=2, levels="1")

,level,pred,lower_bound,upper_bound
50,1,11.903735,-14.386130,38.193600
51,1,26.605282,0.315417,52.895147


In [ ]:
forecaster = ForecasterRnn(model, levels=["1", "2"], lags=3)
forecaster.fit(series, store_in_sample_residuals=True)

# Call the predict method
int_preds = forecaster.predict_interval(steps=2, levels="1")

/home/joaquin/miniconda3/envs/skforecast_17_py12/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 20 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.3498
[[nan nan]
 [nan nan]]
